# **<center><font style="color:rgb(100,109,254)">Module 5: Real-Time Style Transfer Painting Portrait</font> </center>**

<center>
    <img src='https://drive.google.com/uc?export=download&id=1RENayQoiXrIIpuheGqM3G6UOAqVfEj9b'> </center>

## **<font style="color:rgb(134,19,348)"> Module Outline </font>**

The module can be split into the following parts:

- *Lesson 1: Introduction to Neural Style Transfer Theory*

- *Lesson 2: Apply Neural Style Transfer with OpenCV*

- ***Lesson 3:* Build the Final Application** *(This Tutorial)*

**Please Note**, these Jupyter Notebooks are not for sharing; do read the Copyright message below the Code License Agreement section which is in the last cell of this notebook.
-Taha Anwar

Alright, let's get started.

### **<font style="color:rgb(134,19,348)"> Import the Libraries</font>**

First, we will import the required libraries.

In [1]:
import os
import cv2
import math
from time import time
import numpy as np
import mediapipe as mp
from collections import deque
from previous_lesson import (detectHandsLandmarks, applyNeuralStyleTransfer, 
                             recognizeGestures, calculateDistance)

## **<font style="color:rgb(134,19,348)">Initialize the Hands Landmarks Detection Model</font>**

After that, we will have to initialize the **`mp.solutions.hands`** class and then set up the **`mp.solutions.hands.Hands()`** function with appropriate arguments, as we have been doing in the previous modules.

In [2]:
# Initialize the mediapipe hands class.
mp_hands = mp.solutions.hands

# Set up the Hands functions for videos.
hands_videos = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.8)

## **<font style="color:rgb(134,19,348)">Create a Function to apply Neural Style Transfer</font>**

Now we will create a function **`paint()`**, that will simply utilize the hands' landmarks (found by the **`detectHandsLandmarks()`** function, created in a previous module) to calculate the size and location (i.e. at the center of the hands) of a paintbrush (shaped like a flower) and then paint on a `canvas` *(i.e. just an empty black image)*, that will be utilized later to apply Neural Style Transfer only on the painted part of a webcam feed in real-time.

In [3]:
def paint(frame, canvas, results, current_mode, hands_tips_positions):
    '''
    This function will paint with a very creative shape brush (like a flower) on a canvas.
    Args:
        frame:                A frame/image from the webcam feed.
        canvas:               A black image equal to the webcam feed size, to paint on.
        current_mode:         The current mode enabled i.e., either paint or erase.
        hands_tips_positions: A dictionary containing the landmarks of the tips of the fingers of the hands.
    Returns:    
        frame:  The frame with the eraser drawn on it, in case the erase mode was enabled.
        canvas: The black image with the intented drawings on it.
    '''
    
    # Get the height and width of the frame.
    height, width, _ = frame.shape
    
    # Iterate over the found hands in the frame.
    for hand_index, hand_info in enumerate(results.multi_handedness):
        
        # Retrieve the label of the found hand.
        hand_label = hand_info.classification[0].label.upper()
        
        # Retrieve the landmarks of the found hand.
        hand_landmarks =  results.multi_hand_landmarks[hand_index]
        
        # Get the wrist landmarks of the hand, we are iterating upon.
        x1, y1 = (int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x*width),
                  int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y*height))
        
        # Get the middle finger tip landmarks.
        x2, y2 = hands_tips_positions[hand_label]['MIDDLE']
        
        # Calculate the center of the hand, we are iterating upon.
        hand_center_x = (x1+x2)//2
        hand_center_y = (y1+y2)//2
        
        # Calculate the brush size accordingly to the hand size, we are iterating upon.
        brush_size = calculateDistance(frame, point1=hands_tips_positions[hand_label]['PINKY'],
                                       point2=hands_tips_positions[hand_label]['THUMB'], draw=False, display=False)
        
        # Calculate the radius of the brush.
        radius = brush_size//2
        
        # Initialize a list to store the brush (of the hand, we are iterating upon) coordinates.
        brush_pts = []
        
        # Iterate over 20 times.
        # As the brush will have 20 corners and will be shaped like a flower.
        for i in range(20):
            
            # Get the x and y coordinates of a brush corner.
            x = hand_center_x + radius* math.cos(i * 18 * math.pi / 20)
            y = hand_center_y + radius * math.sin(i * 18 * math.pi / 20) 
            
            # Append the coordinates into the list.
            brush_pts.append((x, y))
        
        # Check if the paint mode is enabled.
        if current_mode == 'Paint Mode':
            
            # Draw the shape (brush) with white color, on the canvas.
            canvas = cv2.fillPoly(canvas, pts = [np.array(brush_pts, np.int32)],
                                  color=(255,255,255))
        
        # Check if the erase mode is enabled.
        elif current_mode == 'Erase Mode':
            
            # Draw the shape (brush) with black color, on the canvas.
            # This will erase the paint at the brush location.
            canvas = cv2.fillPoly(canvas, pts = [np.array(brush_pts, np.int32)],
                                  color=(0,0,0))
            
            # Create a copy of the frame.
            frame_copy = frame.copy()
            
            # Draw the shape (brush) with white color, on the copy of the frame.
            # This is drawn just to represent an eraser on the hands locations.
            frame_copy = cv2.fillPoly(frame_copy, pts = [np.array(brush_pts, np.int32)],
                                color=(255,255,255))
            
            # Perform weighted addition to give the eraser a transparency look.
            frame = cv2.addWeighted(frame, 0.5, frame_copy, 0.5, 0)
    
    # Return the frame and the canvas.
    return frame, canvas

Now we will utilize the function **`paint()`** and the `canvas` (it created) to apply neural style transfer only on the intended parts of a webcam feed in real-time. We will be switching between `PAINT` and `ERASE` mode with the hand gestures (`INDEX POINTING UP` to enable paint mode and `VICTORY` to enable erase mode) and with the keys `P` (for paint) and `E` (for erase) as well.

In [7]:
# Initialize the VideoCapture object to read from the webcam.
camera_video = cv2.VideoCapture(0)
camera_video.set(3,1280)
camera_video.set(4,960)

# Initialize a canvas to draw on.
canvas = np.zeros(shape=(int(camera_video.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                         int(camera_video.get(cv2.CAP_PROP_FRAME_WIDTH)), 3),
                  dtype=np.uint8)

# Create named window for resizing purposes.
cv2.namedWindow('Real-Time Style Transfer Painting Portrait', cv2.WINDOW_NORMAL)

# Specify the path of the directory which contains the trained models.
model_folder = 'models'

# Initialize a list to store the loaded models.
models = []

# Iterate over the models files in the directory.
for model_name in os.listdir(model_folder):
    
    # Load a model and append it into the list.
    models.append(cv2.dnn.readNetFromTorch(os.path.join(model_folder,model_name)))

# Initialize a variable to store the index of the selected model.
selected_model_index=0

# Initialize a variable to store the buffer length.
BUFFER_MAX_LENGTH = 2

# Initialize a buffer to store recognized gestures.
buffer = deque([], maxlen=BUFFER_MAX_LENGTH)

# Initialize a variable to store the current mode.
current_mode = None

# Iterate until the webcam is accessed successfully.
while camera_video.isOpened():
   
    # Read a frame.
    ok, frame = camera_video.read()
    
    # Check if the frame is not read properly,
    # then continue to the next iteration to read the next frame.
    if not ok:
        continue
    
    # Flip the frame horizontally for natural (selfie-view) visualization.
    frame = cv2.flip(frame, 1)
    
    # Get the height and width of the frame of the webcam video.
    frame_height, frame_width, _ = frame.shape
    
    # Apply neural style transfer on the frame using the selected model.
    nst_frame = applyNeuralStyleTransfer(frame, models[selected_model_index % len(models)], display=False)
    
    # Perform Hands landmarks detection on the frame.
    frame, results = detectHandsLandmarks(frame, hands_videos, draw=False, display=False)
    
    # Check if the hands landmarks in the frame are detected.
    if results.multi_hand_landmarks:  
        
        # Perform a hand gesture recognition.
        current_gesture, hands_tips_positions = recognizeGestures(frame, results, 'RIGHT', draw=False,
                                                                  display=False)
        # Check if a known gesture is recognized.
        if current_gesture != 'UNKNOWN':
            
            # Check if all the gestures stored in the buffer are equal to the current gesture.
            if all(current_gesture==gesture for gesture in buffer):
                
                # Append the current gesture into the buffer.
                buffer.append(current_gesture)
                
            # Otherwise.
            else:
                
                # Clear the buffer.
                buffer.clear()
            
            # Check if the length of the buffer is equal to the maxlength, that is 20.
            if len(buffer) == BUFFER_MAX_LENGTH:
                
                # Check if the current hand gesture is INDEX POINTING UP.
                if current_gesture == 'INDEX POINTING UP':
                    
                    # Enable the paint mode.
                    current_mode = 'Paint Mode'
                    
                # Check if the current hand gesture is VICTORY.
                elif current_gesture == 'VICTORY':
                    
                    # Enable the erase mode.
                    current_mode = 'Erase Mode'
        
        # Paint or Erase on the canvas depending upon the current mode enabled. 
        frame, canvas = paint(frame, canvas, results, current_mode, hands_tips_positions)
    
    # Update the pixel values of the frame with the nst frame (neural style transfer applied) 
    # values at the indexes where canvas!=0 that is where something is drawn on the canvas.
    #frame[np.mean(canvas, axis=2)!=0] = nst_frame[np.mean(canvas, axis=2)!=0]
    frame[np.mean(canvas, axis=2)!=0] = nst_frame[np.mean(canvas, axis=2)!=0]

    
    # Check if a mode is enabled.
    if current_mode:
        
        # Write the enabled mode on the frame.
        cv2.putText(frame, current_mode + ' Enabled', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (0, 255, 0), 1)
    
    # Write the instructions (to switch modes and neural style) on the frame.
    cv2.putText(frame, 'Press P to Paint', (5, frame_height-10), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, (255, 255, 255), 1)
    
    cv2.putText(frame, 'Press E to Erase', (5, frame_height-30), cv2.FONT_HERSHEY_SIMPLEX, 
                0.5, (255, 255, 255), 1)
    
    cv2.putText(frame, 'Press S to Switch Neural Style', (5, frame_height-50),  cv2.FONT_HERSHEY_SIMPLEX, 
                0.5,(255, 255, 255), 1)
    
    # Display the frame.
    cv2.imshow("canvas", canvas)
    cv2.imshow("Real-Time Style Transfer Painting Portrait", frame)
    cv2.imshow("nst", nst_frame/255)

    
    # Wait for 1ms. If a key is pressed, retreive the ASCII code of the key.
    k = cv2.waitKey(1) & 0xFF
    
     # Check if 'ESC' is pressed and break the loop.
    if(k == 27):
        break
        
    # Check if 's' is pressed then increment the neural style transfer model index.
    elif (k == ord('s')):
        selected_model_index = selected_model_index + 1 
    
    # Check if 'p' is pressed then enable the paint mode.
    elif (k == ord('p')):
        current_mode = 'Paint Mode'
    
    # Check if 'e' is pressed then enable the erase mode.
    elif (k == ord('e')):
        current_mode = 'Erase Mode'
    
    # Check if 'c' is pressed then clear the canvas.
    elif k == ord('c'):
        canvas = np.zeros(shape=(frame_height, frame_width, 3), dtype=np.uint8)

# Release the VideoCapture Object and close the windows.
camera_video.release()
cv2.destroyAllWindows()

Perfect! the application is working as we intended. We can apply Neural Style Transfer anywhere we want with just the movements of our hands in real-time.

### **<font style="color:rgb(255,140,0)"> Code License Agreement </font>**
```
Copyright (c) 2022 Bleedai.com

Feel free to use this code for your own projects commercial or noncommercial, these projects can be Research-based, just for fun, for-profit, or even Education with the exception that you’re not going to use it for developing a course, book, guide, or any other educational products.

Under *NO CONDITION OR CIRCUMSTANCE* you may use this code for your own paid educational or self-promotional ventures without written consent from Taha Anwar (BleedAI.com).

```
